### Simple GEN-AI app using Langchain

Load all the environment variables

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT")

Scrape data from a website

In [11]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

class_label = "beebom-single-content-container"
web_url = "https://beebom.com/valorant-characters-agents-abilities"

loader = WebBaseLoader(web_paths=[web_url],
                bs_kwargs={
                    "parse_only": bs4.SoupStrainer(class_=class_label),
                })
docs = loader.load()

Divide the raw data from the website into chunks

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

splitted_docs = text_splitter.split_documents(docs)

In [13]:
splitted_docs

[Document(metadata={'source': 'https://beebom.com/valorant-characters-agents-abilities'}, page_content='First-person shooter games are always based on aim unless you realize how important utilities are. That is why Valorant’s agent selection is a crucial element to your victory each time you jump into a match. Every Valorant agent has its unique set of abilities and all the characters are important in their own roles. We are talking about all the Valorant characters (updated till Tejo) and their abilities in this guide. You will learn how every agent’s ability works and how they impact the gameplay and mechanics in Valorant. So, let’s learn about all the agents and their abilities to help kickstart your winning streak in Valorant.\nComplete List of Valorant Characters\n\nNote: \nThese characters have been arranged on the basis of their release date. We have first listed the agents that are unlocked by default in Valorant, followed by the characters that were available to unlock at laun

Generate embeddings of the documents

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"
embedder = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    multi_process=True,
    model_kwargs={
        "device": "cuda",
    },
    encode_kwargs={
        "normalize_embeddings": True,
    }
)

Store the embeddings in the vector database FAISS

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore_db = FAISS.from_documents(
    splitted_docs,
    embedder
)

TypeError: FAISS.__init__() got an unexpected keyword argument 'index_name'